# Implementácia konvolúcie a poolingu

Na tomto cvičení implementujeme základné operácie konvolučných neurónových sietí, a to konvolúciu a *pooling*. Programovať budeme v programovacom jazyku Python, ktorý by ste už mali poznať z predošlých predmetov. Pri implementácii použijeme knižnicu `numpy`, ktorá sa používa veľmi často v projektoch s hlbokým učením, a na ňu sú stavané ďalšie knižnice ako `tensorflow` a `pytorch`.

Ak nemáte nainštalovaný Python, najnovšiu verziu si môžete stiahnuť [tu](https://www.python.org/downloads/), odporúčame ale postupovať podľa [kroku 1. tohto manuálu](https://github.com/ianmagyar/dl-course/blob/master/labs/00%20-%20Setting-up-tensorflow.md)

## 1. Štruktúra riešenia

Kostru dnešného riešenia nájdete [tu](resources/lab03/lab03.py) alebo môžete si stiahnuť tento jupyter notebook a pracovať priamo v ňom.

Na začiatku si importujeme knižnicu `numpy` a definujeme niekoľko ukážkových príkladov pre obrázky a filtre (príklady z [minulého cvičenia](lab02-ANNs-and-convolution.ipynb)):

In [1]:
import numpy as np


image1 = np.array([
    [3, 2, 1, 2, 0],
    [0, 4, 2, 0, 1],
    [0, 2, 3, 1, 1],
    [1, 3, 4, 0, 0],
    [2, 1, 2, 1, 0]
])

filter1 = np.array([
    [0, 1, 2],
    [2, 2, 0],
    [0, 1, 2]
])

image2 = np.array([
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
])

filter2 = np.array([
    [1, 2, 1],
    [0, 0, 0],
    [-1, -2, -1]
])

V skripte je deklarovaných šesť funkcií, ktoré budeme implementovať v nasledujúcich krokoch - tri pomocné funkcie a tri ktoré obsahujú implementáciu konvolúcie, max poolingu a average poolingu. Ich podrobný popis nájdete v komentároch, úlohu týchto funkcií ale vieme zhrnúť nasledovne:

- `get_result_array` - vytvorí prázdne n-rozmerné pole s hodnotami inicializovanými na 0. Jej úlohou je vypočítať rozmery výslednej mapy po aplikácii konvolúcie resp. *pooling*u.
- `get_padding_value` - vypočíta potrebný padding na základe parametrov konvolúcie/*pooling*u.
- `get_padded_image` - pridá do originálneho vstupného obrázka *padding* podľa *padding* hodnoty.
- `convolute` - vykoná konvolúciu so zadanými parametrami.
- `maxPool` - vykoná *max pooling* so zadanými parametrami.
- `avgPool` - vykoná *average pooling* so zadanými parametrami.

## 2. `get_result_array`

Pred tým, než sa spustíme do implementácie operácií konvolúcia a *pooling* musíme pochopiť funkciu parametrov *stride* a *padding*.

*Stride* vyjadruje to, o koľko posunieme konvolučný alebo *pooling* filter po jednom kroku výpočtov. Najčastejšie nadobúda hodnotu 1, najmä pri použití *padding*u. Na dnešnom cvičení budeme predpokladať, že jeho hodnota je 1, neskôr svoje riešenie môžete rozšíriť tak, aby podporovalo aj iné hodnoty *stride*.

*Padding* je úprava vstupného obrazu pred aplikáciou konvolúcie alebo *pooling*u. Síce má niekoľko variantov, najčastejšie sa ale používa *zero padding*, teda pridávanie nulových hodnôt na okraje obrázka vo všetkých smeroch. Z možných spôsobov použitia *padding*u dnes implementujeme dva, a to *no padding* (teda na obrázok vôbec nepoužijeme *padding*) a *same padding*, čo znamená, že v každom smere pridáme rovnaký počet núl tak, aby výsledok operácie mal rovnaké rozmery ako vstupný obrázok.

Ak platia vyššie uvedené predpoklady, rozmery výstupnej príznakovej mapy vieme vypočítať podľa vzorca:

\begin{equation*}
m_{d} = \frac{I_{d} \cdot k_{d} + 2 \cdot P}{S} + 1,
\end{equation*}

kde $m$ je mapa, $I$ je obrázok, $k$ je filter (alebo *kernel*), $P$ je *padding* hodnota (počet pridaných núl), $S$ je *stride* a $d$ je rozmer (šírka alebo výška). Keďže ale pri konvolúcii veľmi často používame štvorcové obrazy, rovnica platí pre oba rozmery naraz.

**Implementujte funkciu `get_result_array` tak, aby vrátila n-rozmerné pole hodnôt inicializovaných na 0.** Vstupné parametre funkcie nosia nasledovné hodnoty:

- `image_shape` - dvojica celočíselných hodnôt udávajúca rozmery vstupného obrazu (výška x šírka) ($I$)
- `kernel_shape` - dvojica celočíselných hodnôt udávajúca rozmery vstupného obrazu (výška x šírka) ($k$)
- `stride` - celočíselná hodnota, vyjadruje posun po aplikácii operácie ($S$)
- `P` - *padding* hodnota, celé číslo ($P$)

Vo funkcii by ste mali vykonať nasledovné kroky:

1. vypočítať rozmery výstupnej príznakovej mapy podľa vyššie uvedeného vzorca
2. skontrolujte, či výsledné rozmery sú celé čísla a ak nie, vyhoďte príslušnú chybu
3. funkcia vráti dvojrozmerné pole 0 hodnôt s vypočítanými rozmermi (výška x šírka)

In [2]:
def get_result_array(image_shape, kernel_shape, stride, P):
    return None

## 3. `get_padding_value`

V ďalšom kroku implementujeme funkciu `get_padding_value`, ktorá vypočíta *padding* hodnotu, teda počet pridaných 0 pri danom nastavení parametrov. Ako sme už písali v bode 2, dnes budeme používať iba dva spôsoby *padding*u, a to *no padding* a *same padding*. V rámci funkcie takisto potrebujeme skontrolovať, či nastavenie parametrov dáva platný výsledok operácie pri použití *padding*u.

V prípade *no padding* nepotrebujeme pridať nuly, teda *padding* hodnota je nulová. Pri *same padding* budeme predpokladať, že *stride* je 1, potom *padding* hodnotu vieme vypočítať podľa vzorca (odvodené z vyššie uvedeného vzorca, kde $m = I$):

\begin{equation*}
P_{d} = \frac{k_{d} - 1}{2},
\end{equation*}

kde $P_{d}$ je *padding* hodnota v dimenzií $d$ a $k_{d}$ je rozmer filtra v danej dimenzií. Ak pracujeme so štvorcovým filtrom, *padding* hodnota je rovnaká v oboch smeroch.

**Implementujte funkciu `get_padding_value` tak, aby vypočítala *padding* hodnotu a tiež skontrolovala, či platia všetky obmedzenia hodnôt parametrov:**

- parameter `padding` môže nadobúdať hodnoty `'none'` a `'same'`
- pri *same padding* musí mať parameter `stride` hodnotu `1`.

Funkcia vráti jedno celé číslo a má nasledujúce parametre:

- `kernel_shape` - dvojica celočíselných hodnôt udávajúca rozmery vstupného obrazu (výška x šírka) ($k$)
- `stride` - celočíselná hodnota, vyjadruje posun po aplikácii operácie ($S$)
- `padding` - reťazec s hodnotou `'none'` pre *no padding* alebo `'same'` pre *same padding*

In [3]:
def get_padding_value(kernel_shape, stride, padding):
    return 0

## 4. `get_padded_image`

Ďalšia funkcia vykoná *padding* nad obrázkom, t.j. pridá potrebný počet 0 vo všetkých smeroch, pôvodný obrázok sa nachádza potom v strede rozšíreného obrázka. Funkcia má dva parametre:

- `image` - pôvodný obrázok
- `P` - *padding* hodnota, počet pridaných núl.

**Implementujte funkciu `get_padded_image`, ktorá vráti numpy pole s rozšíreným obrázkom. Pôvodnú premennú nemeňte.**

In [4]:
def get_padded_image(image, P):
    return image

## 5. `convolute`

V tomto kroku implementujeme konvolúciu. Na predspracovanie obrázka a prípravu výstupnej príznakovej mapy vieme použiť už vytvorené pomocné funkcie, našou jedinou úlohou ostáva naplniť pripravenú príznakovú mapu hodnotami aplikáciou konvolúcie. Funkcia má štyri parametre:

- `image` - n-rozmerné pole reprezentujúce vstupný obrázok
- `kernel` - n-rozmerné pole reprezentujúce konvolučný filter
- `stride` - celočíselná hodnota, vyjadruje posun po aplikácii operácie
- `padding` - reťazec s hodnotou `'none'` pre *no padding* alebo `'same'` pre *same padding*

**Implementujte funkciu `convolute` tak, aby vrátila príznakovú mapu. Pri tom nemeňte pôvodné premenné `image` a `kernel`.**

Po implementácii môžete vaše riešenie otestovať na definovaných príkladoch, alebo na vami zvolenom obrázku (opravte hodnoty parametrov v `main` funkcii).

In [7]:
def convolute(image, kernel, stride=1, padding='none'):
    return image

## 6. `maxPool` a `avgPool`

V poslednom kroku nám ostáva implementovať funkcie pre *pooling*. V skripte máte deklarované dve funkcie pre operácie *max pooling* a *average pooling*, ich telo je veľmi podobné riešeniu konvolúcie, jediný rozdiel je vo výpočte hodnôt v príznakovej mape. Obe funkcie majú štyri parametre:

- `image` - n-rozmerné pole reprezentujúce vstupný obrázok
- `kernel_size` - dvojica celočíselných hodnôt udávajúca rozmery vstupného obrázka (výška x šírka)
- `stride` - celočíselná hodnota, vyjadruje posun po aplikácii operácie
- `padding` - reťazec s hodnotou `'none'` pre *no padding* alebo `'same'` pre *same padding*

**Implementujte a otestujte funkcie `maxPool` a `avgPool` tak, aby vrátili príznakovú mapu po aplikácii operácie *pooling*. Hodnoty vstupných premenných pritom nemeňte.**

In [8]:
def maxPool(image, kernel_size, stride=1, padding='none'):
    return image


def avgPool(image, kernel_size, stride=1, padding='none'):
    return image

## Doplňujúce úlohy

1. Ukážkové príklady počítali s čiernobielymi obrázkami. Opravte vaše riešenie tak, aby podporovalo prácu s obrázkami s viacerými farebnými kanálmi.
2. Výpočet *padding* hodnoty v súčasnej forme nie vždy zabezpečuje rovnaké rozmery vstupov a výstupov. Upravte funkciu tak, aby vstupný obrázok rozšírila prioritne vľavo a hore tak, aby výstupná príznaková mapa bola vždy rovnakého rozmeru ako vstupný obrázok.
3. Upravte výpočet *padding* hodnoty tak, aby podporoval aj iné hodnoty *stride* ako 1.

## Príprava programátorského prostredia

Od budúceho cvičenia budeme pracovať s nástrojmi pre hlboké učenie `tensorflow` a `keras`. Práve preto je potrebné, aby ste si pripravili programátorské prostredie pre vývoj hlbokých modelov. Návod na inštaláciu potrebných nástrojov nájdete [tu](https://github.com/ianmagyar/dl-course/blob/master/labs/00%20-%20Setting-up-tensorflow.md).

Ak vaše zadanie chcete urobiť pomocou `pytorch`, návod na inštaláciu je dostupný [tu](https://github.com/ianmagyar/dl-course/blob/master/labs/00%20-%20PyTorch.md).

## Použité zdroje

- [Convolution Image Size, Filter Size, Padding and Stride](https://jamesmccaffrey.wordpress.com/2018/05/30/convolution-image-size-filter-size-padding-and-stride/)